In [1]:
import os
import pandas as pd

In [2]:
#google cloud big query libaries
from google.cloud import bigquery
from google.oauth2 import service_account

key_path = r"C:\Users\bhereth\streetlight-temp-analysis-e2b201d26862.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [3]:
#directories
working_directory = os.getcwd()
data_folder         = os.path.join(working_directory, "data"        )
intermediate_folder = os.path.join(working_directory, "intermediate")
results_folder      = os.path.join(working_directory, "results"     )


# Import Big Query Data

In [4]:
#ease of use
daytype0 = '0: All Days (Mo-Su)'
daytype1 = '1: Weekday (Tu-Th)'
daytype2 = '2: Weekend Day (Sa-Su)'
dataper1 = '1. All year'
dataper2 = '2. Sep-Nov'
dataper3 = '3. Dec-Feb'
dataper4 = '4. Mar-May'
dataper5 = '5. Jun-Aug'
daypart0 = '0: All Day (12am-12am)'
daypart1 = '1: Early AM (12am-6am)'
daypart2 = '2: Peak AM (6am-9am)'
daypart3 = '3: Mid-Day (9am-3pm)'
daypart4 = '4: Peak PM (3pm-6pm)'
daypart5 = '5: Late PM (6pm-12am)'

#zones for interzonal definition
dImportTables = [
                    ['ut-udot-adap-dev.streetlight_data.udot_commercial_all_year_2019'],
                    ['ut-udot-adap-dev.streetlight_data.udot_commercial_fall_2019'    ],
                    ['ut-udot-adap-dev.streetlight_data.udot_commercial_spring_2019'  ],
                    ['ut-udot-adap-dev.streetlight_data.udot_commercial_summer_2019'  ],
                    ['ut-udot-adap-dev.streetlight_data.udot_commercial_winter_2019'  ]
                ]

sJoinTable = "ut-udot-adap-dev.streetlight_data.wfrc_streetlight_taz"

# Create the pandas DataFrame
dfImportTables = pd.DataFrame(dImportTables, columns = ['tableName'])
display(dfImportTables)


,tableName
0,ut-udot-adap-dev.streetlight_data.udot_commerc...
1,ut-udot-adap-dev.streetlight_data.udot_commerc...
2,ut-udot-adap-dev.streetlight_data.udot_commerc...
3,ut-udot-adap-dev.streetlight_data.udot_commerc...
4,ut-udot-adap-dev.streetlight_data.udot_commerc...


In [5]:
#create query of all subareaid to subareaid truck flows using join to taz table with subareaid field
dfTruckDataBySubareaID = pd.DataFrame()

for dit in dImportTables:
    sql = """
    SELECT
        orig_table.SUBAREAID AS origin_subareaid,
        dest_table.SUBAREAID AS destination_subareaid,
        od_table.mode_of_travel AS mode_of_travel,
        od_table.vehicle_weight AS vehicle_weight,
        od_table.day_type AS day_type,
        od_table.day_part AS day_part,
        od_table.data_period AS data_period,
        SUM(od_table.o_d_traffic_sample_trip_counts) AS od_trip_counts_sum,
        SUM(od_table.o_d_traffic_calibrated_trip_volume) AS od_trip_volume_sum,
        COUNT(mode_of_travel) AS num_records
    FROM
        (""" + dit[0] + """ as od_table
            LEFT JOIN
                """ + sJoinTable + """ AS orig_table
                ON od_table.origin_zone_name = orig_table.SL_COTAZID)
                    LEFT JOIN """ + sJoinTable + """ AS dest_table
                    ON od_table.destination_zone_name = dest_table.SL_COTAZID
    GROUP BY
        orig_table.SUBAREAID,
        dest_table.SUBAREAID,
        od_table.mode_of_travel,
        od_table.vehicle_weight,
        od_table.day_type,
        od_table.day_part,
        od_table.data_period
    """
    #display(sql)

    dfQuery = client.query(sql).to_dataframe()
    dfTruckDataBySubareaID = dfTruckDataBySubareaID.append(dfQuery)

display(dfTruckDataBySubareaID)



,origin_subareaid,destination_subareaid,mode_of_travel,vehicle_weight,day_type,day_part,data_period,od_trip_counts_sum,od_trip_volume_sum,num_records
0,1.0,NaN,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,27419,2289.573822,2491
1,NaN,0.0,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,20364,1700.458854,2041
2,1.0,1.0,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,100925,8427.558917,36365
3,NaN,1.0,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,20923,1747.137134,2910
4,NaN,5.0,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,4532,378.436433,293
...,...,...,...,...,...,...,...,...,...,...
1653,3.0,2.0,Trucks,Heavy,2: Weekend Day (Sa-Su),3: Mid-Day (9am-3pm),3. Dec-Feb,1,0.524400,1
1654,4.0,5.0,Trucks,Medium,2: Weekend Day (Sa-Su),3: Mid-Day (9am-3pm),3. Dec-Feb,1,0.972000,1
1655,5.0,2.0,Trucks,Medium,2: Weekend Day (Sa-Su),2: Peak AM (6am-9am),3. Dec-Feb,1,0.972000,1
1656,2.0,4.0,Trucks,Medium,2: Weekend Day (Sa-Su),1: Early AM (12am-6am),3. Dec-Feb,1,0.972000,1


In [6]:
#read totals without the join to check against
dfTruckDataSumForChecks = pd.DataFrame()

for dit in dImportTables:

    sqlsumforchecks = """ 
    SELECT
        SUM(od_table.o_d_traffic_sample_trip_counts) AS od_trip_counts_sum,
        SUM(od_table.o_d_traffic_calibrated_trip_volume) AS od_trip_volume_sum,
        COUNT(od_table.mode_of_travel) AS num_records
    FROM
        """ + dit[0] + """ AS od_table
    """

    #display(sqlsumforchecks)

    dfQuerySum = client.query(sqlsumforchecks).to_dataframe()
    dfQuerySum['table'] = dit[0]
    dfTruckDataSumForChecks = dfTruckDataSumForChecks.append(dfQuerySum)

dfTruckDataSumForChecks

,od_trip_counts_sum,od_trip_volume_sum,num_records,table
0,19711308,1.769780e+06,5237876,ut-udot-adap-dev.streetlight_data.udot_commerc...
0,4594646,1.749065e+06,1736809,ut-udot-adap-dev.streetlight_data.udot_commerc...
0,5449188,1.906250e+06,1924193,ut-udot-adap-dev.streetlight_data.udot_commerc...
0,5452558,1.933118e+06,1992275,ut-udot-adap-dev.streetlight_data.udot_commerc...
0,4214916,1.489832e+06,1615426,ut-udot-adap-dev.streetlight_data.udot_commerc...


In [7]:
display(dfTruckDataSumForChecks.sum())
display(dfTruckDataBySubareaID.sum())

od_trip_counts_sum                                             39422616
od_trip_volume_sum                                       8848046.049443
num_records                                                    12506579
table                 ut-udot-adap-dev.streetlight_data.udot_commerc...
dtype: object

origin_subareaid                                                   17375.0
destination_subareaid                                              17395.0
mode_of_travel           TrucksTrucksTrucksTrucksTrucksTrucksTrucksTruc...
vehicle_weight           HeavyHeavyHeavyHeavyHeavyHeavyHeavyHeavyHeavyH...
day_type                 1: Weekdays (Tu-Th)1: Weekdays (Tu-Th)1: Weekd...
day_part                 2: Peak AM (6am-9am)2: Peak AM (6am-9am)2: Pea...
data_period              1. All year1. All year1. All year1. All year1....
od_trip_counts_sum                                                39422616
od_trip_volume_sum                                          8848046.049434
num_records                                                       12506579
dtype: object

# Large District

In [8]:
#create query of all subareaid to subareaid truck flows using join to taz table with subareaid field
dfTruckDataByDISTLRG = pd.DataFrame()

for dit in dImportTables:
    sql = """
    SELECT
        orig_table.DISTLRG AS origin_distlrg,
        dest_table.DISTLRG AS destination_distlrg,
        od_table.mode_of_travel AS mode_of_travel,
        od_table.vehicle_weight AS vehicle_weight,
        od_table.day_type AS day_type,
        od_table.day_part AS day_part,
        od_table.data_period AS data_period,
        SUM(od_table.o_d_traffic_sample_trip_counts) AS od_trip_counts_sum,
        SUM(od_table.o_d_traffic_calibrated_trip_volume) AS od_trip_volume_sum,
        COUNT(mode_of_travel) AS num_records
    FROM
        (""" + dit[0] + """ as od_table
            LEFT JOIN
                """ + sJoinTable + """ AS orig_table
                ON od_table.origin_zone_name = orig_table.SL_COTAZID)
                    LEFT JOIN """ + sJoinTable + """ AS dest_table
                    ON od_table.destination_zone_name = dest_table.SL_COTAZID
    GROUP BY
        orig_table.DISTLRG,
        dest_table.DISTLRG,
        od_table.mode_of_travel,
        od_table.vehicle_weight,
        od_table.day_type,
        od_table.day_part,
        od_table.data_period
    """
    #display(sql)

    dfQuery = client.query(sql).to_dataframe()
    dfTruckDataByDISTLRG = dfTruckDataByDISTLRG.append(dfQuery)

dfTruckDataByDISTLRG

,origin_distlrg,destination_distlrg,mode_of_travel,vehicle_weight,day_type,day_part,data_period,od_trip_counts_sum,od_trip_volume_sum,num_records
0,2.0,NaN,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,16034,1338.890064,839
1,1.0,NaN,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,19515,1629.564650,1577
2,1.0,1.0,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,26551,2217.093057,9687
3,11.0,12.0,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,2304,192.391338,535
4,NaN,12.0,Trucks,Heavy,1: Weekdays (Tu-Th),2: Peak AM (6am-9am),1. All year,5729,478.389745,543
...,...,...,...,...,...,...,...,...,...,...
15047,1.0,22.0,Trucks,Medium,2: Weekend Day (Sa-Su),1: Early AM (12am-6am),3. Dec-Feb,33,32.076000,28
15048,14.0,4.0,Trucks,Medium,2: Weekend Day (Sa-Su),2: Peak AM (6am-9am),3. Dec-Feb,33,32.076000,27
15049,16.0,1.0,Trucks,Medium,2: Weekend Day (Sa-Su),2: Peak AM (6am-9am),3. Dec-Feb,33,32.076000,31
15050,4.0,8.0,Trucks,Medium,2: Weekend Day (Sa-Su),3: Mid-Day (9am-3pm),3. Dec-Feb,33,32.076000,32


In [9]:
display(dfTruckDataByDISTLRG.sum())
display(dfTruckDataSumForChecks.sum())
display(dfTruckDataBySubareaID.sum())

origin_distlrg                                                  939514.0
destination_distlrg                                             946004.0
mode_of_travel         TrucksTrucksTrucksTrucksTrucksTrucksTrucksTruc...
vehicle_weight         HeavyHeavyHeavyHeavyHeavyHeavyHeavyHeavyHeavyH...
day_type               1: Weekdays (Tu-Th)1: Weekdays (Tu-Th)1: Weekd...
day_part               2: Peak AM (6am-9am)2: Peak AM (6am-9am)2: Pea...
data_period            1. All year1. All year1. All year1. All year1....
od_trip_counts_sum                                              39422616
od_trip_volume_sum                                        8848046.049435
num_records                                                     12506579
dtype: object

od_trip_counts_sum                                             39422616
od_trip_volume_sum                                       8848046.049443
num_records                                                    12506579
table                 ut-udot-adap-dev.streetlight_data.udot_commerc...
dtype: object

origin_subareaid                                                   17375.0
destination_subareaid                                              17395.0
mode_of_travel           TrucksTrucksTrucksTrucksTrucksTrucksTrucksTruc...
vehicle_weight           HeavyHeavyHeavyHeavyHeavyHeavyHeavyHeavyHeavyH...
day_type                 1: Weekdays (Tu-Th)1: Weekdays (Tu-Th)1: Weekd...
day_part                 2: Peak AM (6am-9am)2: Peak AM (6am-9am)2: Pea...
data_period              1. All year1. All year1. All year1. All year1....
od_trip_counts_sum                                                39422616
od_trip_volume_sum                                          8848046.049434
num_records                                                       12506579
dtype: object

# Export to CSV

In [10]:
#export to CSV
dfTruckDataBySubareaID.to_csv(os.path.join(intermediate_folder, r"TruckDataBySubAreaID.csv"))
dfTruckDataByDISTLRG.to_csv(os.path.join(intermediate_folder, r"TruckDataByDISTLRG.csv"))
dfTruckDataSumForChecks.to_csv(os.path.join(intermediate_folder, r"TruckDataSumForChecks.csv"))